# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 21.8 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO,solutions
import cv2
from google.colab.patches import cv2_imshow

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
model = YOLO("YOLO_Model.pt")

In [5]:
model.info()

Model summary: 365 layers, 68,156,460 parameters, 0 gradients, 258.1 GFLOPs


(365, 68156460, 0, 258.13867519999997)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [6]:
cap_video = cv2.VideoCapture('/content/YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [7]:
height=int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
height

1080

In [8]:
width=int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
width

1920

In [9]:
fps=int(cap_video.get(cv2.CAP_PROP_FPS))
fps

29

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [10]:
output_path="object_counting_output.avi"

In [11]:
codec = cv2.VideoWriter_fourcc(*"mp4v")

In [12]:
video_writer = cv2.VideoWriter(output_path, codec, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [13]:
roi_top = (20, 0, width - 20, height // 4)
top_count = 0

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [14]:
while cap_video.isOpened():
    success, im0 = cap_video.read()
    if not success:
        print("Video processing completed.")
        break

    results = model(im0)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = result.names[int(box.cls)]
            confidence = box.conf[0]

            cv2.rectangle(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(im0, f'{label} {confidence:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            if roi_top[1] <= center_y <= roi_top[3] and roi_top[0] <= center_x <= roi_top[2]:
                top_count += 1


    cv2.putText(im0, f'Top: {top_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(im0, 'Nouf', (width - 150, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.rectangle(im0, (roi_top[0], roi_top[1]), (roi_top[2], roi_top[3]), (0, 255, 255), 2)
    video_writer.write(im0)

cap_video.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 16 cars, 137.5ms
Speed: 18.7ms preprocess, 137.5ms inference, 1202.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 cars, 62.8ms
Speed: 4.3ms preprocess, 62.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 59.0ms
Speed: 3.1ms preprocess, 59.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 50.0ms
Speed: 3.0ms preprocess, 50.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 48.3ms
Speed: 2.4ms preprocess, 48.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 40.4ms
Speed: 3.8ms preprocess, 40.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 40.4ms
Speed: 3.1ms preprocess, 40.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 35.4ms
Speed: 3.1ms preprocess, 35.4ms inference, 1.2ms postprocess per imag

## Save and Submit Your Work

Submit both the notebook and the output video